In [ ]:
import pandas as pd


In [ ]:
tt_filepath = "./train_test.csv"
v_filepath = "./validation.csv"
ex_vali_filepath = './ex.csv'
train_test = pd.read_csv(tt_filepath)
vali = pd.read_csv(v_filepath)
ex_vali = pd.read_csv(ex_vali_filepath)

In [ ]:
data_t =  pd.concat([train_test[["label"]],train_test.loc[:,'L_R':"PET_original_gldm_SmallDependenceLowGrayLevelEmphasis"]],axis=1)
data_v = pd.concat([vali[["label"]],vali.loc[:,'L_R':"PET_original_gldm_SmallDependenceLowGrayLevelEmphasis"]],axis=1)
data_ex = pd.concat([ex_vali[["label"]],ex_vali.loc[:,'L_R':"PET_original_gldm_SmallDependenceLowGrayLevelEmphasis"]],axis=1)

In [ ]:
import numpy as np
import sklearn
data_t = sklearn.utils.shuffle(data_t)
data_v = sklearn.utils.shuffle(data_v)
data_ex =sklearn.utils.shuffle(data_ex)

X = data_t.loc[:,"L_R":]
y = data_t['label']
Xvv = data_v.loc[:,"L_R":]
y_v = data_v["label"]
columns = Xvv.columns
X_ex = data_ex.loc[:,"L_R":]
y_ex = data_ex['label']
X_ex.head(5)

In [ ]:
feature =[
    'shortest_distance',
    'L_R','minor_axis_tumor','SUVmax',
    'PET_original_shape_Elongation',
    'CTR',
    'original_shape_Maximum2DDiameterColumn',
    'major_axis_tumor',
    'original_shape_Flatness',
    'PET_original_firstorder_Minimum',
    'original_firstorder_10Percentile',
    'original_firstorder_Range',
    'original_glcm_DifferenceVariance',
    'original_glcm_Imc2',
    'original_glrlm_RunLengthNonUniformity']

In [ ]:
X_v = Xvv[feature]
X_t = X[feature]
X_ex = X_ex[feature]

In [ ]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegressionCV
import lightgbm as lgb
import xgboost as xgb
import warnings
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot

In [ ]:
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('rf', RandomForestClassifier()))
    level0.append(('svm', SVC()))
    level0.append(('dbt', AdaBoostClassifier()))
    # level0.append(('knn', KNeighborsClassifier()))
    # level0.append(('nb', GaussianNB()))
    # level0.append(('bnb',BernoulliNB()))
    # level0.append(('dtree',DecisionTreeClassifier()))
    level0.append(('gtree',GradientBoostingClassifier()))
    # level0.append(('mlp',MLPClassifier()))
    level0.append(('gbm',lgb.LGBMClassifier(verbose=-1)))
    level0.append(('xgb',xgb.XGBClassifier()))
    level0.append(('lr',LogisticRegressionCV()))
    # define meta learner model
    level1 = MLPClassifier()  #能到0.76

    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model


In [ ]:
def get_models():
    models = dict()
    models['rf'] = RandomForestClassifier()
    models['svm'] = SVC()
    models['dbt'] = AdaBoostClassifier()
    models['knn'] = KNeighborsClassifier()
    models['nb'] = GaussianNB()
    models['bnb'] =BernoulliNB()
    models['dtree'] =DecisionTreeClassifier()
    models['gtree'] =GradientBoostingClassifier()
    models['mlp'] =MLPClassifier()
    models['gbm'] =lgb.LGBMClassifier()
    models['xgb'] =xgb.XGBClassifier()
    models['lr'] =LogisticRegressionCV()
    models['stacking'] = get_stacking()
    return models

In [ ]:
def evaluate_model(model,x,y):
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)
    scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [ ]:
import warnings 
warnings.filterwarnings("ignore")
models = get_models()
for name, model in models.items():
    model_v = model.fit(X_t,y)
    score_v = model_v.score(X_v,y_v)
    y_pred = model_v.predict(X_v)
    y_probs = model_v.predict_proba(X_v)
    score_ex = model_v.score(X_ex,y_ex)
    y_pred_ex = model_v.predict(X_ex)
    y_probs_ex = model_v.predict_proba(X_ex)
    print(classification_report(y_v, y_pred))
    print(confusion_matrix(y_ex, y_pred_ex))
    print(classification_report(y_ex, y_pred_ex))

In [ ]:
models = get_models()
for name, model in models.items():
    model_v = model.fit(X_t,y)


In [ ]:
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model,X_t,y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()